# TP - ES Problèmes inverses

Vincent Duval, Nicolas Desassis

## Exercice 4 - Déconvolution en traitement d'images
Le but de cette partie est d'étudier l'importance de régulariser un problème inverse avec ou sans bruit.


In [ ]:
using Images, Plots, TestImages
using Wavelets, FFTW
include("toolBoxImage.jl");

**0.** Charger une image. On pourra consulter la liste des images disponibles sur https://testimages.juliaimages.org/

In [ ]:
img=Float32.(Gray.(testimage("lighthouse"))) # house,coffee, chelsea... voir la liste sur https://testimages.juliaimages.org/
M,N=size(img)
plot(Gray.(img),showaxis=false,title="Image originale")

## I. Convolution
On étudie l'opérateur de convolution périodique $A\colon L^2([0,1]^2) \rightarrow L^2([0,1]^2)$ défini par 
 $$\forall x \in [0,1]^2, \quad Au(x)=\int_{[0,1]^2}h(x-s)u(s)\mathrm{d} s, $$
où $h \in L^2([0,1]^2)$ est le noyau de convolution (étendu par 1-périodicité), aussi appelé fonction d'étalement du point (*Point Spread Function*, PSF) ou réponse impulsionnelle.

**I.1)** Rappeler une décomposition en valeurs singulières de l'opérateur $A$.


**I.2)** Pour différentes valeur de fréquences de coupures et types de filtres (Gaussian, Cauchy), observer:
- la réponse impulsionnelle, $h(x)$ pour $x \in [-1/2,1/2]^2$,
- la fonction de transfert, $c_k(h) = \int_{[0,1]^2} h(x)e^{-2\imath \left(k_1 x_1 + k2 x_2 \right)}\mathrm{d}x_1\mathrm{d}x_2 \approx \frac{1}{MN}\sum_{\substack{0\leq j_1\leq M-1,\\0\leq j_2\leq N-1}}h\left(\frac{j_1}{M},\frac{j_2}{N}\right)e^{-2\imath \pi \left(k_1\frac{j_1}{M}+ k_2\frac{j_2}{N}\right)}$,
- l'effet de A sur u.

D'après vous, quels sont les problèmes les plus mal posés?

In [ ]:
f0h=40 # frequence de coupure du filtre (en hauteur - height) # A MODIFIER
f0w=40 # frequence de coupure du filtre (en largeur - width) # A MODIFIER
filtertype ="Cauchy" # Type de filtre ("Gaussian" ou "Cauchy") # A MODIFIER
h,ckh,freqh,freqw=prepare_filter(M,N,f0h,f0w,filtertype) # renvoie la réponse impulsionnelle et sa transformée de Fourier

xh,xw=((1:M).-floor(M/2))/M, ((1:N).-floor(N/2))/N # vecteur position (xheight et xwidth)
display(plot(xh,xw,Gray.(h/maximum(h)),title="Réponse impulsionnelle h"))
display(plot(freqh,freqw,Gray.(fftshift(ckh)),title="Fonction de transfert c_k(h)"))


ckimg=fft(img) # Transformée de Fourier de u
y=real.(ifft(ckh.*ckimg)) # convolution
display(plot(Gray.(y),showaxis=false,title="Signal filtré y=Au"))

On suppose aussi que l'observation a été contaminée par un bruit, $\tilde{y}=y + \delta y$.
Plusieurs types de bruit peuvent contaminer les images. 
Pendant le processus d’acquisition, les capteurs CCD (Charge-Coupled-Devices) sont soumis au
bruit de grenaille, qui est dû à la nature discrète de la quantité mesurée (c’est-à-dire le nombre de photons reçus), et qui suivent une loi de Poisson. L’effet du bruit de grenaille est d’autant plus important que
le nombre de photons est faible, de sorte que les scènes insuffisamment éclairées apparaissent souvent
bruitées. Le bruit thermique est aussi responsable de la dégration des images : en fonction de la température du capteur l’agitation thermique des électrons induit un courant qui s’ajoute au signal original.


Pour simplifier, on prendra ici une perturbation $\delta y$ qui suit une loi gaussienne centrée d'écart type $\nu$.

In [ ]:
nu=0.01# écart-type du bruit # A MODIFIER
yBruit=y+nu*randn(M,N)
ckyBruit= fft(yBruit)
plot(Gray.(yBruit),showaxis=false)

  ## II. Inversion directe
  
  **II.1)** La solution de $Au=y$ est elle unique ? Montrer que cette solution (ou celle de norme minimale le cas échéant) vérifie
  $$c_k(u) = \frac{1}{c_k(h)}c_k(y)$$ 
  et la calculer sur les différents exemples.

In [ ]:
g(x)=1/x  
ckinv= g.(ckh);

In [ ]:
# Reconstruction à partir de l'observation sans bruit
# Essayer avec différentes valeurs de filtrage (voir à la question 2). Par exemple avec gaussian et f0w=80 puis f0w=40 
cky=fft(y)
u=real.(ifft(cky.*ckinv))
plot(Gray.(u),showaxis=false)

In [ ]:
# Reconstruction à partir de l'image bruitée. Idem, essayer plusieurs valeurs de filtres et de bruit
#ubruit=  # A COMPLETER

plot(Gray.(uBruit),showaxis=false)


## III. Filtrage de Tikhonov

On veut régulariser en utilisant la méthode de Tikhonov,
$$ \min_u \frac{\lambda}{2}\|u\|^2 + \frac{1}{2}\|Au-y\|^2$$ 
 (ou bien $\tilde{y}$ remplacera $y$ si l'observation est bruitée).

**III.1)** Montrer que la solution est donnée par 
$$c_k(u) =  \frac{c_k(h)}{\lambda + |c_k(h)|^2} c_k(y).$$
Appliquer ce filtrage sur l'observation $y$ et l'observation bruitée $\tilde{y}$.

In [ ]:
lambda=0.05

# gTik(x)=  # A COMPLETER
ckTik= gTik.(ckh)

# Reconstruction sans bruit
# Essayer avec différentes valeurs de filtrage. Par exemple avec gaussian et f0w=80 puis f0w=40 
uTik=real.(ifft(cky.*ckTik))
plot(Gray.(uTik),showaxis=false)

In [ ]:
# Idem avec du bruit
uBruitTik=real.(ifft(fft(yBruit).*ckTik))
display(plot(Gray.(uBruitTik),showaxis=false))
display(plot(Gray.(yBruit),showaxis=false)) # Décommenter pour comparer

## IV. Troncature spectrale


On veut utiliser la troncature spectrale.
$$ c_k(u) = \begin{cases}\frac{1}{c_k(h)}c_k(y)& \mbox{si $k \in K$,}\\ 0 & \mbox{sinon.}  \end{cases}$$

Ici, en dimension 2, on n'a pas d'ordre naturel sur les fréquences, on ne va pas prendre les $N$ premières, mais plutôt on considèrera des ensembles de la forme

$$ K= \{k \in \mathbb{Z}^2 | \|k\|\leq r \}.$$

**IV.1)** On fait la troncature spectrale en prenant une fréquence maximale inférieur à un certaine valeur:
$$ K_{\infty}(r) = \{ k \in \mathbb{Z}^2 | |k_1|\leq r, |k_2|\leq r\},$$

In [ ]:
r=80.0
ckTronc=zeros(ComplexF64,M,N)
Finf=[max(abs(freqh[i]),abs(freqw[j])) for i in 1:M, j in 1:N]
Kinf=findall(Finf .<=r)    # Ne pas oublier le "." pour tester la condition en chaque point
ckTronc[Kinf] .= cky[Kinf]./(ckh[Kinf])
uTronc=real.(ifft(ckTronc))
plot(Gray.(uTronc),showaxis=false)

In [ ]:
# Faire de même avec l'observation bruitée:
ckTroncBruit=zeros(ComplexF64,M,N)
# ckTroncBruit[Kinf] .=  # A COMPLETER
uTroncBruit=real.(ifft(ckTroncBruit))
plot(Gray.(uTroncBruit),showaxis=false)

**IV.2)** On prend la norme euclidienne sur les fréquences:
$$ K_{2}(r) = \{ k \in \mathbb{Z}^2 | |k_1|^2+|k_2|^2\leq r^2\}.$$

In [ ]:
r=80.0
ckTronc=zeros(ComplexF64,M,N)
# F2=  # A COMPLETER
K2=findall(F2 .<=r^2)    # Ne pas oublier le "." pour tester la condition en chaque point
ckTronc[K2] .= cky[K2]./(ckh[K2])
uTronc=real.(ifft(ckTronc))
plot(Gray.(uTronc),showaxis=false)

In [ ]:
# Faire de même avec l'observation bruitée:
ckTroncBruit=zeros(ComplexF64,M,N)
# ckTroncBruit[K2] .=  # A COMPLETER
uTroncBruit=real.(ifft(ckTroncBruit))
plot(Gray.(uTroncBruit),showaxis=false)

## V. Régularisation par norme de Sobolev

On considère à la place une autre régularisation pour Tikhonov,

$$ \min_u \frac{\lambda}{2}\|Du\|_{\ell^2}^2 + \frac{1}{2}\|Au-y\|^2$$
où $D$ est un opérateur de gradient discret (avec conditions de bord périodiques, pour simplifier):
$$\forall j \in \{1,\ldots,N\},\quad(Du)_{i,j,1}=\frac{1}{M}\times\begin{cases} u_{i+1,j}-u_{i,j} &\mbox{pour $1\leq i \leq M-1$}, \\
                u_{1,j}-u_{M,j} &\mbox{pour $i=M$}.\end{cases}
                $$
$$\forall i \in \{1,\ldots,M\},\quad (Du)_{i,j,2}=\frac{1}{N}\times\begin{cases} u_{i,j+1}-u_{i,j} &\mbox{pour  $1\leq j \leq N-1$, } \\
                u_{i,1}-u_{i,N} &\mbox{pour $j=N$.}\end{cases}
                $$

**V.0) [à faire à la maison, hors TP]** Montrer que le terme de régularisation $\|Du\|_{\ell^2}^2=\sum_{i,j}\left[ ((Du)_{i,j,1})^2+((Du)_{i,j,2})^2 \right]$ peut se réécrire dans le domaine de la transformée Fourier discrète (en écrivant $k=(k_1,k_2))$:
$$\|Du\|_{\ell^2}^2= 4\sum_{\substack{0\leq k_1\leq M-1\\0\leq k_2\leq N}} |c_k|^2\alpha_k^2$$

où 
$$\alpha_k^2= M^2\sin^2\left(\frac{k_1\pi}{M}\right)+N^2\sin^2\left(\frac{k_2\pi}{N}\right). $$


**V.1)** En déduire que les solutions du problème de régularisation sont données par:
$$c_k(u) =\frac{c_k(h)}{4\lambda\alpha_k^2 + |c_k(h)|^2}c_k(y).$$

In [ ]:
lambda=0.000001


alpha2=[(M*sin(fh/M))^2 + (N*sin(fw/N))^2 for fh in freqh, fw in freqw]

# gTik(x)=  # A COMPLETER= gTik.(ckh)
ckSob= ckh./(4*lambda*alpha2 .+ abs2.(ckh)) # TODO ICI
# Reconstruction sans bruit
# Essayer avec différentes valeurs de filtrage. Par exemple avec gaussian et f0w=80 puis f0w=40 
uSob=real.(ifft(cky.*ckSob))
plot(Gray.(uSob),showaxis=false)

In [ ]:
uSobBruit=real.(ifft(ckyBruit.*ckSob))
display(plot(Gray.(uSobBruit)))
display(plot(Gray.(yBruit)))